In [1]:
import os 
os.environ["ISISROOT"] = "/usgs/cpkgs/anaconda3_linux/envs/isis3.9.0"

from matplotlib import pyplot as plt

from plio.io.io_gdal import GeoDataset 
import numpy as np

from autocnet.cg import change_detection as cd
from autocnet.utils.utils import bytescale 

from plio.io.io_gdal import GeoDataset
from pysis import isis
from pysis.exceptions import ProcessError

In [2]:
roi1_proj = "/scratch/krodriguez/hirise/ESP_031022_1415_RED1.stitched.norm.34517_37165.proj.cub"
roi2_proj = "/scratch/krodriguez/hirise/ESP_014038_1415_RED0.stitched.norm.30421_33069.proj.cub"

In [3]:
ssa = GeoDataset('out.cub')

In [4]:
ssa_array = ssa.read_array()
ssa_slice = ssa_array[700:1200,1100:1500]

In [5]:
roi1_proj_geo = GeoDataset(roi1_proj)
roi2_proj_geo = GeoDataset(roi2_proj)
isis_null = pysis.specialpixels.SPECIAL_PIXELS['Real']['Null']

proj_array1 = roi1_proj_geo.read_array()
proj_array2 = roi2_proj_geo.read_array()

proj_array1[proj_array1 == isis_null] = np.nan
proj_array2[proj_array2 == isis_null] = np.nan

plt.figure(0, figsize=(10, 10))
plt.imshow(proj_array1, alpha=.4)
plt.imshow(proj_array2, alpha=.4)

NameError: name 'pysis' is not defined

In [ ]:
slice1 = proj_array1[700:1200,1100:1500]
slice2 = proj_array2[700:1200,1100:1500]

In [ ]:
def im_diff(arr1,arr2):
    diff_array = arr2 - arr1
    diff_array[np.isnan(diff_array)] = 0
    diff_array = bytescale(diff_array)
    return diff_array

In [ ]:
t1,t2 = cd.blob_detector(slice1, slice2, ssa_slice, im_diff, max_sigma=5, num_sigma=3, threshold=.25)

In [ ]:
fig = plt.figure(0, figsize=(30, 30))
axis = fig.add_subplot(1,1,1)
plt.imshow(im_diff(slice1,slice2), alpha=1)
for pt1, pt2 in t1:
    y, x, r = pt1
    c = plt.Circle((x, y), r, color='red', linewidth=2, fill=False)
    axis.add_patch(c)
    y, x, r = pt2
    c = plt.Circle((x, y), r, color='yellow', linewidth=2, fill=False)
    axis.add_patch(c)
    
for pt1, pt2 in t2:
    y, x, r = pt1
    c = plt.Circle((x, y), r, color='orange', linewidth=2, fill=False)
    axis.add_patch(c)
    y, x, r = pt2
    c = plt.Circle((x, y), r, color='blue', linewidth=2, fill=False)
    axis.add_patch(c)